# Leonardo's Revolving Bridge: Rapid Deployment

> *"I have plans for bridges, very light and strong and suitable for carrying very easily..."*  
> — Leonardo da Vinci, Letter to Ludovico Sforza

## Introduction
Leonardo designed a rotating bridge for military use. It could be packed up, transported, and quickly swung across a river. The key innovation was a **counterweight system** (often using water) to balance the bridge as it rotated.

In this notebook, we analyze:
1.  **Static Equilibrium**: Balancing the bridge weight with a counterweight.
2.  **Structural Integrity**: Ensuring the bridge doesn't break under load.
3.  **Deployment**: The dynamics of swinging the bridge into place.

---

In [ ]:
# Install the davinci-codex library
!pip install -q git+https://github.com/Shannon-Labs/davinci-codex.git

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Set "Brutalist Academic" plotting style
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'text.color': 'black',
    'axes.labelcolor': 'black',
    'xtick.color': 'black',
    'ytick.color': 'black'
})


## The Physics Model

### 1. Moment Balance
For the bridge to rotate easily, the moment (torque) from the bridge span must be balanced by the counterweight:
$$ M_{bridge} = W_{bridge} \cdot L_{cg\_bridge} $$
$$ M_{counter} = W_{counter} \cdot L_{arm} $$
Ideally, $M_{bridge} \approx M_{counter}$.

### 2. Beam Deflection
We model the bridge as a cantilever beam during rotation:
$$ \delta_{max} = \frac{w L^4}{8 E I} $$
Where $w$ is load per unit length, $L$ is length, $E$ is Young's Modulus, and $I$ is the moment of inertia.

In [ ]:
def analyze_bridge(span_length_m, counterweight_mass_kg, load_kg):
    # Constants
    g = 9.81
    E_wood = 11e9 # Oak, Pa
    beam_width = 0.3
    beam_height = 0.4
    I = (beam_width * beam_height**3) / 12
    
    # Bridge Properties
    linear_density = 50.0 # kg/m (structure)
    bridge_mass = linear_density * span_length_m
    bridge_weight = bridge_mass * g
    
    # Moments (about pivot)
    # Bridge CG is at L/2
    M_bridge = bridge_weight * (span_length_m / 2)
    
    # Counterweight arm (fixed)
    arm_length = 3.0 # m
    M_counter = (counterweight_mass_kg * g) * arm_length
    
    net_moment = M_bridge - M_counter
    
    # Deflection (Cantilever + Point Load at tip)
    # Distributed load from self-weight
    w = linear_density * g
    deflection_self = (w * span_length_m**4) / (8 * E_wood * I)
    
    # Point load from payload (at tip for worst case)
    P = load_kg * g
    deflection_load = (P * span_length_m**3) / (3 * E_wood * I)
    
    total_deflection = deflection_self + deflection_load
    
    # Stress
    M_max = (w * span_length_m**2 / 2) + (P * span_length_m)
    stress = (M_max * (beam_height / 2)) / I
    yield_stress = 40e6 # 40 MPa for Oak
    safety_factor = yield_stress / stress
    
    return net_moment, total_deflection, safety_factor

def plot_bridge_analysis(span, cw_mass, payload):
    moment, deflection, safety = analyze_bridge(span, cw_mass, payload)
    
    # Visualization
    fig, ax = plt.subplots(figsize=(10, 4))
    
    # Draw Pivot
    ax.plot(0, 0, 'ko', markersize=10)
    ax.plot([0, 0], [-1, 0], 'k-', linewidth=3)
    
    # Draw Bridge
    # Deflected shape (exaggerated)
    x = np.linspace(0, span, 100)
    # Simple parabolic approximation for visual
    y = - (deflection / span**2) * x**2
    ax.plot(x, y, 'b-', linewidth=4, label='Bridge Span')
    
    # Draw Counterweight
    ax.plot([-3, 0], [0, 0], 'k-', linewidth=4)
    ax.plot(-3, 0, 'rs', markersize=15, label='Counterweight')
    
    # Draw Payload
    ax.plot(span, y[-1], 'g^', markersize=10, label='Payload')
    
    ax.set_xlim(-5, span + 2)
    ax.set_ylim(-deflection * 2 - 1, 2)
    ax.set_aspect('equal')
    ax.set_title(f'Bridge State (Safety Factor: {safety:.2f})')
    ax.legend()
    
    plt.show()
    
    print(f"Net Moment: {moment:.0f} Nm ({'Balanced' if abs(moment)<1000 else 'Unbalanced'})")
    print(f"Tip Deflection: {deflection*1000:.1f} mm")
    print(f"Safety Factor: {safety:.2f} {'(SAFE)' if safety > 2.0 else '(WARNING)'}")

interact(plot_bridge_analysis, 
         span=widgets.FloatSlider(min=5.0, max=20.0, step=1.0, value=10.0, description='Span (m)'),
         cw_mass=widgets.IntSlider(min=100, max=5000, step=100, value=1000, description='CW Mass (kg)'),
         payload=widgets.IntSlider(min=0, max=2000, step=100, value=500, description='Payload (kg)'));

## Conclusion

The analysis shows the critical importance of the **counterweight**. Without it, the moment at the pivot would be enormous, making rotation impossible for a human crew. 

Leonardo's use of a water tank for the counterweight was ingenious because it allowed the mass to be adjusted (by adding or removing water) to perfectly balance the bridge, regardless of variations in timber weight.